<a href="https://colab.research.google.com/github/jyPark1105/ML_DL_Study_Myself/blob/main/Chap%203-2_Linear%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3-2 : Linear Regression

### Introduction

        *k-최근접 이웃 회귀와 선형 회귀 알고리즘의 차이를 이해하고 여러 가지 선형 회귀 모델 생성하기
        
**농어의 길이: 50cm, 농어의 무게: 1.5kg -> 앞서 만든 모델들을 사용할 수 있을까?**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/ML_DL_Alone')

%load_ext autoreload
%autoreload 2

### The Limitation of k-Nearest Neighbor algorithm

    Introduction의 문제를 먼저 재현하기
    3-1절에서 사용한 데이터와 모델 준비

In [ ]:
# 농어 입력 데이터 준비
import numpy as np

perch_length = np.array(
    [ 8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0,
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5,
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5,
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0,
     36.0, 36.5, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0,
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
    )

perch_weight = np.array(
    [  5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0,
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0,
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0,
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0,
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0,
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0,
     1000.0, 1000.0]
    )

In [ ]:
# 데이터를 훈련 세트와 테스트 세트로 나누기 + 특성 데이터는 2차원 배열로 변환
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트로 나누기
train_input, test_input, train_target, test_target = train_test_split(perch_length, perch_weight, random_state=42)

# 2차원 배열로 변환
train_input = train_input.reshape(-1, 1)
test_input = test_input.reshape(-1, 1)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# 최근접 이웃 개수를 3으로 하는 회귀 모델의 객체 생성
knr = KNeighborsRegressor(n_neighbors=3)

# 모델 훈련
knr.fit(train_input, train_target)

In [ ]:
# length = 50cm인 농어의 무게를 예측하기
print(knr.predict([[50]]))

perch_length[0] == 8.4cm

perch_length[-1] == 44.0cm

* 현재 농어 길이의 범위는 8.4cm ~ 44.0cm
* 농어의 길이가 증가할수록 농어의 무게가 증가하는 경향으로 모델을 예측
* 50cm라는 농어의 길이는 당연히 길이가 44.0cm인 농어의 무게보다 커야함

    그러나 최근접 이웃의 개수 3개의 무게를 평균하여 50cm 농어의 무게로서 예측하면 예상보다 훨씬 낮은 값이 예측됨


**산점도를 그려서 데이터 형태를 파악해보자**

In [ ]:
# Train set, 50cm 농어, 이 농어의 최근접 이웃을 산점도에 표시

import matplotlib.pyplot as plt

# 50cm 농어의 이웃들을 구하기
distances, indexes = knr.kneighbors([[50]])

# Train set 산점도 그리기
plt.scatter(train_input, train_target)

# Train set 중 이웃 샘플 산점도 그리기
plt.scatter(train_input[indexes], train_target[indexes], marker='D')

# 50cm 농어 데이터
plt.scatter(50, 1033, marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 50cm 농어 데이터의 이웃 샘플들의 평균값 출력
print(np.mean(train_target[indexes]))

*New sample 값이 Train set의 범위를 벗어나면 엉뚱한 값을 예측할 수 있음

    ex) length = 100cm -> weight_pred = 1033.0g
    (이웃 샘플 3개가 동일하므로)


In [ ]:
print(knr.predict([[100]]))
print(knr.predict([[200]]))

In [ ]:
distances_100, indexes_100 = knr.kneighbors([[50]])

# Train set 산점도 그리기
plt.scatter(train_input, train_target)

# Train set 중 이웃 샘플 산점도 그리기
plt.scatter(train_input[indexes_100], train_target[indexes_100], marker='D')

# 100cm 농어 데이터
plt.scatter(100, 1033, marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

    농어의 길이가 아무리 커도 무게가 더 이상 늘어나지 않을 것
    k-최근접 이웃을 사용하여 이 문제를 해결하기 위해서는 가장 큰 농어가 포함되도록 Train set를 다시 만들어야함

k-Nearest Neighbor algorithm 말고 다른 방법은 존재하지 않는 것인가?

-> **Linear Regression**


### **Linear Regression**

* 자주 사용되는 대표적인 회귀 알고리즘
* 비교적 간단하고 성능이 뛰어남
* "# of features = 1"인 경우 최적의 직선을 학습하는 알고리즘

    
    결정계수 R^2 value에 대해
        1. 직선의 위치가 훈련 세트의 평균와 가깝다면 R^2 = 0 (모든 농어의 무게를 하나로 예측)
        2. R^2 < 0 (농어의 무게를 완전히 반대로 예측)
        3. R^2 > 0 (농어의 무게를 적절히 예측)

**Scikit-learn**

-> sklearn.linear_model package 아래에 LinearRegression class로 선형 회귀 알고리즘을 구현해놓음

Package: sklearn.linear_model

Class: LinearRegression

________________________________________
**Tip**

    Scikit-learn의 모델 클래스들은 훈련, 평가, 예측하는 메서드명이 모두 동일.
    즉, fit(), score(), predict()로서 모델을 생성하고 예측.

In [ ]:
# 선형 회귀 모듈 임포트
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# Train
lr.fit(train_input, train_target)

# 50cm 농어에 대한 예측
print(lr.predict([[50]]))

# 100cm 농어에 대한 예측
print(lr.predict([[100]]))

LinearRegression Class
1. 선형 회귀 모델이 학습한 **하나의 최적의 직선을 그리기 위해 기울기와 절편이 필요**
2. y = a * x + b form

   x = 농어의 길이, y = 농어의 무게

LinearRegression Class가 찾은 **a와 b는 객체의 coef_와 intercept_ 속성에 저장**

    coef_ : 기울기를 종종 계수(coefficient) 혹은 가중치(weight)라고 부름


In [ ]:
# 최적의 기울기와 절편 파악
print(lr.coef_, lr.intercept_)

**모델 기반 학습**

1. coef_와 intercept_를 **model parameter**라고 함
2. 머신러닝 알고리즘의 훈련 과정 = 최적의 모델 파라미터를 찾는 과정 -> 모델 기반 학습

**사례 기반 학습**

1. KNN 알고리즘의 경우 모델 파라미터가 존재하지 않음
2. 훈련 세트를 저장하는 것이 전부 -> 사례 기반 학습

plt.plot(x, y) : x축과 y축의 값을 받아 선 그래프를 그리는 함수

In [ ]:
# perch_length: 15 ~ 50까지 산점도 그리기

## model parameters
a = lr.coef_
b = lr.intercept_

## Train set 산점도 그리기
plt.scatter(train_input, train_target)

## 15 ~ 50까지 선형 그래프 그리기
plt.plot([15, 50], [a*15+b, a*50+b])

## 50cm 농어 데이터
plt.scatter(50, 1241.8, marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

1. 위 코드 실행 후 직선 -> 선형 회귀 알고리즘이 훈련 데이터셋을 통해 학습한 후 얻은 최적의 직선

2. 길이 50cm 농어에 대한 예측 -> 직선의 연장선에 존재

       이제 훈련 세트 범위를 벗어난 농어의 무게도 예측 가능
        -> 훈련 세트와 테스트 세트의 R^2 값을 확인

In [ ]:
print(lr.score(train_input, train_target))
print(lr.score(test_input, test_target))

# 모델이 훈련 세트에 과대적합했다고 볼 수 없는 점수
## 오히려 과소적합에 빠진 모델이라고 볼 수 있음

### Polynomial Regression(다항 회귀)

    직감적으로 그래프에서 무언가 이상한 점을 느낌
    1. 선형 회귀가 만든 직선이 왼쪽 아래로 쭉 뻗어있음 -> 이대로 예측하면 농어의 무게 < 0g (불가능한 일)
    2. 농어의 길이와 무게에 대한 산점도는 일직선이 아닌 위로 조금 구부러진 곡선 형태에 가까움 -> 최적의 직선이 아닌 최적의 곡선을 찾기

이런 2차 곡선 형태를 그리기 위해 x^2(길이의 제곱) 항이 훈련 세트에 추가되어야 함

**Numpy 배열을 사용하여 2차 곡선 모델 간단히 만들기**: np.column_stack() 메서드 활용

In [ ]:
# np.column_stack((훈련 제곱값, 훈련 값))
train_poly = np.column_stack((train_input**2, train_input))
test_poly = np.column_stack((test_input**2, test_input))

print(train_poly.shape, test_poly.shape)

In [ ]:
# Squared Train set 확인
print(train_poly)
print()

# Squared Test set 확인
print(test_poly)

train_poly를 활용하여 선형 회귀 모델을 다시 훈련

    1. 이 모델이 2차 방정식 y = a * x^2 + b * x + c의 최적의 a, b, c 값을 찾아줄 것으로 기대
    2. 주목할 점은 data만 제곱항을 추가하고, target은 그대로 사용 -> 목표하는 값(y)은 어떤 그래프를 훈련하든지 바꿀 필요 x
    3. 2차 방정식의 모델 훈련 후 50cm 농어 데이터에 대해 predict -> 이 때 모델 y = a * x^2 + b * x + c에 x^2과 x값을 전달
    4. fit(_poly, _target) 형태로 훈련

In [ ]:
# 2차식 형태의 모델 객체 생성
lr_poly = LinearRegression()

# 모델 훈련
lr_poly.fit(train_poly, train_target)

# 50cm 농어 데이터에 대한 예측
print(lr_poly.predict([[50**2, 50]]))

In [ ]:
# Model parameters 출력
print(lr_poly.coef_, lr_poly.intercept_)

# y = (1.01) * x^2 + (-21.6) * x + 116.5

2차 방정식 = 선형 회귀?

    길이의 제곱 -> 왕길이로 치환하면 무게 = 1.01 * 왕길이 + (-21.6) * 길이 + 116.5 형태의 선형 방정식으로 나타낼 수 있음
    즉, 무게는 왕길이와 길이의 선형 관계식으로 표현 가능 -> 다항식(Polynomial)

    Polynomial을 사용한 Linear regression -> "Polynomial Regression"

최적의 모델 파라미터 a, b, c를 구했으니 산점도 그리기(1씩 짧게 끊어서 그리기)


In [ ]:
# 최적의 모델 파라미터들
a, b, c = 1.01, -21.6, 116.5

# 구간별 직선 그리기 위해 15 ~ 49까지 정수 배열 만들기
point = np.arange(15, 50)

# 훈련 세트의 산점도 그리기
plt.scatter(train_input, train_target)

# 15 ~ 49까지 2차 방정식 그래프 그리기
plt.plot(point, a*(point**2) + b * point + c)

# 50cm 농어 데이터
plt.scatter(50, 1574.2, marker='^')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# Overfitting, Underfitting 여부 확인
print(lr_poly.score(train_poly, train_target))
print(lr_poly.score(test_poly, test_target))

정리
 1. Train set와 Test set에 대한 점수가 크게 높아짐
 2. 그러나 Test score > Train score -> "Underfitting"

우리는 둘 다 높은 점수를 가지면서 Train score가 Test score보다 높아야 함 -> 조금 더 복잡한 모델이 필요


### 문제 해결 과정

**선형 회귀로 훈련 세트 범위 밖의 샘플 예측**

    1. KNN 회귀 알고리즘: 훈련 세트 범위 밖의 샘플에 대한 예측이 불가능(아무리 멀리 떨어져 있어도 가장 가까운 훈련 샘플의 타깃을 평균하여 예측)

       --> "Linear Regression"

    Linear Regression
    2. 이 문제를 해결하기 위해 선형 회귀 알고리즘을 도입(Scikit-learn의 LinearRegression 클래스를 사용하면 KNN 알고리즘과 동일한 방식으로 훈련 및 예측 가능)

    3. 최적의 기울기와 절편(Model parameters)을 구해 최적의 모델을 구축하는 것.(객체명: lr)
       실제로 y = a * x + b에서 a = lr.coef_와 b = lr.intercept_ 객체의 속성으로서 옵티멀한 모델 파라미터 값을 구할 수 있음
    
    4. 선형 회귀는 KNN 회귀와 달리 훈련 데이터 밖의 샘플에 대한 예측도 Good!
       그러나 농어의 무게를 음수로 예측하는 "단순한 모델 형태"
       
       --> "Polynomial Regression"
    
    Polynomial Regression
    5. 농어의 길이(x)를 제곱한 항을 추가하여 훈련 세트에 추가한 다음 선형 회귀 모델을 다시 훈련시키는 방식

    6. 2차 방정식의 그래프 형태를 학습, 훈련 세트가 분포된 형태를 잘 표현함

    7. 단순한 선형 회귀보다 훈련 세트와 테스트 세트의 성능이 증가 -> 그러나 아직 Test score > Train score이므로 Underfitting의 경향 O

3-3절: 조금 더 복잡한 모델을 만들어 Underfitting 문제 해결 및 너무 복잡한 모델을 만들어서 생기는 Overfitting을 억제하는 방법에 대해 배우기

## 마무리

1. Keywords
    * **Linear Regression** : 특성과 타깃 사이의 관계를 가장 잘 나타내는 최적의 선형 방정식을 찾는 모델.

    특성이 하나면 직선 방정식 형태가 됨

    * **Model Parameter** : 모델이 훈련 데이터에서 학습한 파라미터(혹은 계수, 가중치), 머신러닝에서 종종 가중치는 방정식의 기울기 및 절편을 의미하는 경우 多

    * **Polynomial Regresion** : 다항식을 사용하여 특성과 타깃 사이의 관계를 나타내는 선형 회귀. 비선형 표현이지만 선형 회귀로 표현 가능

        y = a * x^2 + b * x + c에서 x^2 -> x'으로 치환하면 y = a * x' + b * x + c의 선형 방정식 형태. 이 때 x'과 x는 모델이 훈련할 데이터이므로 모두 모델에 합쳐서(column_stack) 전달(fit), 이후 예측(predict(poly, target) 형태)

2. 핵심 package와 function
    * scikit-learn
     * LinearRegression: 사이킷런의 선형 회귀 클래스

        1. 기울기 a = coef_ -> 특성에 대한 계수를 포함한 배열.
           배열의 크기: # of features
        2. intercept_ 속성에는 절편이 저장되어 있음. 또한 fit_intercept 매개변수를 False로 지정 시 절편 학습 x(기본값=True)